In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import math
from sklearn.model_selection import train_test_split
from scipy import stats
from keras import backend as K
from keras.callbacks import TensorBoard, EarlyStopping
from sklearn.model_selection import StratifiedKFold
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Dropout
from keras.layers import TimeDistributed
from keras.layers import Activation
from keras.layers import MaxPooling1D, UpSampling1D
from keras.layers import MaxPooling2D
from keras.layers import Flatten, Reshape
from keras.layers import Input, Dense, Bidirectional
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import Conv2D
from keras.layers import RepeatVector
from keras.layers.pooling import GlobalAveragePooling1D, GlobalMaxPooling1D
from keras.optimizers import SGD
from keras.utils import to_categorical
from keras.utils import plot_model
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from keras.models import Model
from keras.models import Sequential
import numpy as np
import os
import matplotlib.pyplot as plt
import random as rand
import pandas as pd
import tensorflow
import keras
import statistics as st
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation
from keras.optimizers import SGD
from keras.utils.vis_utils import plot_model
from sklearn.preprocessing import LabelEncoder
from collections import Counter

In [ ]:
os.chdir("/Users/carterloftus/Documents/ECS 289/")

In [ ]:
import p_f1_recall as metric

results = []

In [ ]:
### HYPERPARAMETERS ##
batch_size, epochs, verbose = 200, 100, 1

group_size = 48
BiD = False
LSTM_nodes = 50
encoding_dim = 32


In [ ]:
##### First proces unlabled data

data_u = pd.read_csv('fullest_unlabeled_data.csv')

### Remove unncessary columns
data_u = data_u.iloc[:,[3,4,5,7,8,9,10]]

### Standardize the features
def standardize(data,col_nums_to_standardize):
    for col in col_nums_to_standardize:
        data.iloc[:,col] = (data.iloc[:,col] - np.mean(data.iloc[:,col]))/np.std(data.iloc[:,col])
    return data

data_u = standardize(data_u,col_nums_to_standardize = range(6))


#### Now process the labeled data
data_train = pd.read_csv('final_training.csv')
data_test = pd.read_csv('final_testing.csv')

### Remove unncessary columns
data_train = data_train.iloc[:,[3,4,5,7,8,9,10,12]]  ## Add 13 if need to keep IDs ## Drop heading because of low mutual information
data_test = data_test.iloc[:,[3,4,5,7,8,9,10,12]]

### Standardize the features
data_train = standardize(data_train,col_nums_to_standardize = range(6))
data_test = standardize(data_test,col_nums_to_standardize = range(6))

In [ ]:
#Break features into groups of size n

def break_into_samples(data,group_size):
    l = []
    sep_ts = set(data.loc[:,'ID'])
    for ts in sep_ts:
        sub_data  = np.array(data.loc[data.loc[:,'ID']== ts,:])
        a = int(sub_data.shape[0]/group_size)
        if a != 0:
            sub_data = sub_data[:(a*group_size),:]
            sub_data = [np.split(sub_data,a)]
            l = l + sub_data
    data = np.vstack(l)
    data = data[:,:,:-1]
    return data

chunked_data_train = break_into_samples(data_train,group_size)
chunked_data_test = break_into_samples(data_test,group_size)
chunked_u_data = break_into_samples(data_u,group_size)

In [ ]:
def split_train_val(data,labeled = True,thresh = 0.8):
    np.random.shuffle(data)
    thresh1 = int(data.shape[0]*thresh)
    training = data[:thresh1,:,:]
    val = data[thresh1:,:,:]
    return training, val


global_num_classes = -1

def split_feat_lab(data,many_to_one = True, prop = 0.65):
    features = data[:,:,:data.shape[2]-1]
    labels = data[:,:,data.shape[2]-1]
    if many_to_one == True:
        y_temp = np.zeros(labels.shape[0])
        y_temp.fill(np.nan)
        for i in range(labels.shape[0]):
            row = labels[i,:]
            trans_class = (Counter(row)).most_common(1)[0]
            if trans_class[1] < group_size*prop : 
                 y_temp[i] = 4
            else:
                 y_temp[i] = st.mode(row)
        labels = y_temp
        global global_num_classes
    if global_num_classes < 0:
        global_num_classes = len(set(labels.flatten('C')))
    num_classes = global_num_classes
    labels = to_categorical(labels,num_classes)
    return features,labels

In [ ]:
chunked_data_train1, chunked_data_val = split_train_val(chunked_data_train)
training, val = split_train_val(chunked_u_data)

train_x,train_y = split_feat_lab(chunked_data_train1,many_to_one=False)
val_x,val_y = split_feat_lab(chunked_data_val,many_to_one=False)
test_x,test_y = split_feat_lab(chunked_data_test,many_to_one=False)

In [ ]:

n_timesteps = training.shape[1]
n_features = training.shape[2]

def build_autoencoder(bi_directional = BiD):
    raw_input = Input(shape=(n_timesteps,n_features))
    if bi_directional == True:
        layer_1 = BiDirectional(LSTM(LSTM_nodes, return_sequences = True))(raw_input)
    else:
        layer_1 = LSTM(LSTM_nodes, return_sequences = True)(raw_input)
    layer_2 = Dropout(0.5)(layer_1)
    layer_3 = TimeDistributed(Dense(encoding_dim*2, activation='relu'))(layer_2)
    encoded = TimeDistributed(Dense(encoding_dim, activation='relu'))(layer_3)
    decoding = TimeDistributed(Dense(encoding_dim*2, activation='relu'))(encoded)
    decoding_2 = Dropout(0.5)(decoding)
    if bi_directional == True:
        decoding_3 = BiDirectional(LSTM(LSTM_nodes, return_sequences = True))(decoding_2)
    else:
        decoding_3 = LSTM(LSTM_nodes, return_sequences = True)(decoding_2)
    output = TimeDistributed(Dense(n_features))(decoding_2)
    encoder = Model(raw_input,encoded)
    autoencoder = Model(raw_input,output)
    return autoencoder, encoder
    
autoencoder, encoder = build_autoencoder()

init_weights = encoder.get_weights()

def autoencoder_train(autoencoder):
    autoencoder.compile(optimizer='adam', loss='mse')
    autoencoder.fit(training, training,
                epochs=epochs,
                batch_size=batch_size,
                shuffle=True,
                validation_data=(val, val),
                callbacks=[EarlyStopping(monitor='val_loss', min_delta=0, patience=3, verbose=0, mode='auto', baseline=None, restore_best_weights=True)])
    loss = autoencoder.evaluate(val, val, batch_size=batch_size, verbose=verbose)
    return loss

autoencoder_loss = autoencoder_train(autoencoder)

trained_weights = encoder.get_weights()

In [ ]:
encoder.set_weights(trained_weights)
new_layer = TimeDistributed(Dense(global_num_classes, activation='softmax'))(encoder.output)
pre_trained = Model(encoder.input,new_layer)
pre_trained.summary()

def train_model(model):
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy',metric.recall,metric.precision,metric.f1])
    model.fit(train_x,train_y,epochs=epochs,batch_size=batch_size,verbose=verbose,validation_data=(val_x,val_y),
             callbacks=[EarlyStopping(monitor='val_loss', min_delta=0, patience=3, verbose=0, mode='auto', baseline=None, restore_best_weights=True)])
    loss,accuracy,recall_score,precision_score,f_score = model.evaluate(val_x, val_y, batch_size=batch_size, verbose=verbose)
    return loss,accuracy,recall_score,precision_score,f_score

m1_loss,m1_accuracy,m1_recall_score,m1_precision_score,m1_f_score = train_model(pre_trained)

In [ ]:
encoder.set_weights(init_weights)
new_layer = TimeDistributed(Dense(global_num_classes, activation='softmax'))(encoder.output)
no_pre_training = Model(encoder.input,new_layer)
no_pre_training.summary()

m2_loss,m2_accuracy,m2_recall_score,m2_precision_score,m2_f_score = train_model(no_pre_training)

In [ ]:
results.append([autoencoder_loss,m1_f_score,m2_f_score])
print(np.mean(results,axis=0))
print(np.sd(results,axis=0))